In [1]:
import jieba
import random
import warnings
from itertools import chain
warnings.filterwarnings('ignore')

In [2]:
def check_lanuage(string):
    # conclude if the language of string is Chinese
    for char in string:
        if char>='\u4e00' and char<='\u9fa5':
            return True
    return False
def conclude_pattern(string):
    if(string.startswith('?')):
        if (len(string)>2):
            case1=string[1:].startswith('*') and all(char.encode('utf-8').isalpha() for char in string[2:])
            case2=all(char.encode('utf-8').isalpha() for char in string[1:])
            return case1 or case2
        if(len(string)==2):
            return all(char.encode('utf-8').isalpha() for char in string[1:])
    return False

def split_words(string):
    output_words=[]
    if not check_lanuage(string):
         output_words=string.split()
    else:
        temp_chars=''
        for word in jieba.cut(string):
            if word=='?':
                temp_chars+=word
            elif word=='*': 
                if len(temp_chars)==1:
                    temp_chars+=word
            elif word.encode('utf-8').isalpha(): 
                if(len(temp_chars)>=1):
                    temp_chars+=word
            else:
                if(len(temp_chars)>=2):
                    output_words.append(temp_chars)
                    temp_chars=''
                output_words.append(word)
        if(len(temp_chars)>=2):
            output_words.append(temp_chars)
    return output_words

In [3]:
def conclude_same(string1,string2):
    if string1=='' and string2=='':
        return True
    if conclude_pattern(string1[0]):
        return True
    if string1[0]==string2[0]:
        return conclude_same(string1[1:],string2[1:])
    return False

def get_same_start_point(rule_string,saying_string):
    if not rule_string:
        return len(saying_string)
    for index,char in enumerate(saying_string):
        if(char==rule_string[0]):
            if(conclude_same(rule_string[1:],saying_string[index+1:])):
                return index
    return None
def pattern_match_result(rule:list,saying:list):
    if not rule and not saying:
        return []
    string,rest=rule[0],rule[1:]
    if conclude_pattern(string):
        index=get_same_start_point(rest,saying)
        string=string.replace('?*','?')
        return [(string,saying[:index])]+pattern_match_result(rule[1:],saying[index:])
    if string==saying[0]:
        return pattern_match_result(rule[1:],saying[1:])
    else:
        print('not match')
def match_result_to_dict(result:list):
    output_dict={}
    if not result:
        return output_dict
    _,wordlists=zip(*result)
    language=any(check_lanuage(word) for word in chain.from_iterable(wordlists))
    for key,word_list in result:
        output_dict[key]=' '.join(word_list) if not language else ''.join(word_list)
    return output_dict
def pattern_match_conclude(rule:list,saying:list):
    if not rule and not saying:
        return True
    string,rest=rule[0],rule[1:]
    if conclude_pattern(string):
        index=get_same_start_point(rest,saying)
        if index!=None:
            if '*' not in string and index!=1:return False
            return pattern_match_conclude(rule[1:],saying[index:])
        else:
            return False
    if string==saying[0]:
        return pattern_match_conclude(rule[1:],saying[1:])
    else:
        return False
def pattern_match(rule:list,saying:list):
    if pattern_match_conclude(rule,saying):
        return pattern_match_result(rule,saying)
    #print("{} can't be pattern matched to {}".format(rule,saying))
    return []

In [4]:
rule_string1=split_words('你?P今天?*X过的好吗？?*Y')
print(rule_string1)
saying_string1=split_words('你忘记问今天朋友过的好吗？或许你是对的')
print(saying_string1)
print(match_result_to_dict(pattern_match(rule_string1,saying_string1)))
print('******************************************************')
rule_string2=split_words('你?*P今天?*X过的好吗？?*Y')
print(rule_string2)
saying_string2=split_words('你忘记问今天朋友过的好吗？或许你是对的')
print(saying_string2)
print(match_result_to_dict(pattern_match(rule_string2,saying_string2)))
print('******************************************************')
rule_string3=split_words('?*x you ?*y')
print(rule_string3)
saying_string3=split_words('where are you from?')
print(saying_string3)
print(match_result_to_dict(pattern_match(rule_string3,saying_string3)))
print('******************************************************')
rule_string4=split_words('This is ?*x to ?y this ?*Z')
print(rule_string4)
saying_string4=split_words('This is my first time to face this annoying problem')
print(saying_string4)
print(match_result_to_dict(pattern_match(rule_string4,saying_string4)))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/home/stu_18701958249/.local/lib/python3.7/site-packages/jieba/__init__.py", line 154, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmpp5jynhjq' -> '/tmp/jieba.cache'
Loading model cost 0.882 seconds.
Prefix dict has been built successfully.


['你', '?P', '今天', '?*X', '过', '的', '好', '吗', '？', '?*Y']
['你', '忘记', '问', '今天', '朋友', '过', '的', '好', '吗', '？', '或许', '你', '是', '对', '的']
{}
******************************************************
['你', '?*P', '今天', '?*X', '过', '的', '好', '吗', '？', '?*Y']
['你', '忘记', '问', '今天', '朋友', '过', '的', '好', '吗', '？', '或许', '你', '是', '对', '的']
{'?P': '忘记问', '?X': '朋友', '?Y': '或许你是对的'}
******************************************************
['?*x', 'you', '?*y']
['where', 'are', 'you', 'from?']
{'?x': 'where are', '?y': 'from?'}
******************************************************
['This', 'is', '?*x', 'to', '?y', 'this', '?*Z']
['This', 'is', 'my', 'first', 'time', 'to', 'face', 'this', 'annoying', 'problem']
{'?x': 'my first time', '?y': 'face', '?Z': 'annoying problem'}


In [5]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],}

In [6]:
def get_response_from_saying(saying:str,rule_responses:dict):
    language=check_lanuage(saying)
    rules=list(rule_responses.keys())
    saying_list=split_words(saying)
    max_nums=0
    max_rule=rules[0]
    best_result=[]
    for rule in rules:
        rule_list=split_words(rule)
        result=pattern_match(rule_list,saying_list)
        if len(result)>max_nums:
            max_nums=len(result)
            max_rule=rule
            best_result=result
    if best_result:
        pattern_result=match_result_to_dict(best_result)
        best_response=random.choice(rule_responses[max_rule])
        return ' '.join([pattern_result.get(word,word) for word in split_words(best_response)]) if not language else ''.join([pattern_result.get(word,word) for word in split_words(best_response)])
    print('{} can\'t be pattern matched to any pattern of answer mode designed in advance'.format(saying))
    return ''

In [7]:
for _ in range(5):
    print(get_response_from_saying(saying='为啥我想要学习',rule_responses=rule_responses))
print('******************************************************')
print(get_response_from_saying(saying='为啥我知道要学习',rule_responses=rule_responses))
print('******************************************************')
for _ in range(5):
    print(get_response_from_saying(saying='Do you know that I was little sad a moment ago',rule_responses=rule_responses))

你看为啥像学习不
为啥想问你，你觉得学习有什么意义呢
我看你就像学习
你看为啥像学习不
我看你就像学习
******************************************************
为啥我知道要学习 can't be pattern matched to any pattern of answer mode designed in advance

******************************************************
Perhaps I already knew you were little sad a moment ago
Perhaps I already knew you were little sad a moment ago
Why do you tell me you were little sad a moment ago now?
Perhaps I already knew you were little sad a moment ago
Perhaps I already knew you were little sad a moment ago
